In [1]:
import pandas as pd

In [2]:
df = pd.read_excel('data/국민건강보험공단_환경성질환(천식) 의료이용정보_20221231.xlsx')

In [7]:
df.head(3)

STD_YYYYMM   SIDO  CNT
0      200601  서울특별시  103
1      200601  서울특별시   65
2      200601  서울특별시   10

In [4]:
df = df.drop(columns=['SGG','GEN','AGG'])

In [5]:
sido_mapping = pd.DataFrame({
    '시도코드': [11, 26, 27, 28, 29, 30, 31, 36, 41, 42, 43, 44, 45, 46, 47, 48, 50],
    '시도명': ['서울특별시', '부산광역시', '대구광역시', '인천광역시', '광주광역시', '대전광역시', '울산광역시', '세종특별자치시', '경기도', '강원도', '충청북도', '충청남도', '전라북도', '전라남도', '경상북도', '경상남도', '제주특별자치도']
})

In [6]:
df = pd.merge(df, sido_mapping, left_on='SIDO', right_on='시도코드')

# 시도명으로 'SIDO' 열을 대체하고 불필요한 열 제거
df['SIDO'] = df['시도명']
df.drop(['시도코드', '시도명'], axis=1, inplace=True)

In [8]:
# 'STD_YYYYMM'과 'SIDO'를 기준으로 그룹화하고, 'CNT'의 합계를 계산하여 'CNT_SUM' 열에 추가
df['CNT_SUM'] = df.groupby(['STD_YYYYMM', 'SIDO'])['CNT'].transform('sum')
# 날짜별과 시도를 기준으로 and로 해서 카운트수를 합침 

In [9]:
df = df.drop_duplicates(subset=['SIDO', 'STD_YYYYMM', 'CNT_SUM'])
# 시도와 날짜를 기준으로 카운트수를 합쳤기 때문에 
# 중복된 데이터를 제거하면 데이터가 정렬됨 

In [10]:
df.reset_index(drop=True, inplace=True)
# 데이터정렬후 리셋인덱스

In [11]:
df.tail()

STD_YYYYMM     SIDO  CNT  CNT_SUM
3463      202208  제주특별자치도  142     1752
3464      202209  제주특별자치도  315     2458
3465      202210  제주특별자치도  425     2886
3466      202211  제주특별자치도  209     2210
3467      202212  제주특별자치도  257     2289

In [12]:
df = df.drop(columns=['CNT'])

In [13]:
# 비염환자수로 콜롬명 변경
df.rename(columns={'CNT_SUM': '천식환자수'}, inplace=True)

In [14]:
df['천식환자수'].dtype

dtype('int64')

In [15]:
df.to_csv('data/test/천식.csv', index=False)